In [1]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import time
import logging
log = logging.getLogger(__name__)

In [11]:
with open('/Users/okdamava/Downloads/twitter_stock_reactions-main/bearer_token.txt') as fp:
    bearer_token = fp.read()
    
dtformat = '%Y-%m-%dT%H:%M:%SZ'
    
def get_data(tweet):
    data = {
        'id': tweet['id'],
        'created_at': tweet['created_at'],
        'text': tweet['text']
    }
    return data

def gather_tweets(search_term,start_time,end_time):
    endpoint = 'https://api.twitter.com/2/tweets/search/all'
    headers = {'authorization': f'Bearer {bearer_token}'}
    params = {
    'max_results': '500',
    'tweet.fields':'created_at,lang'
    }
    params['query'] = search_term
    params['end_time'] = end_time
    params['start_time'] = start_time
    
    response = requests.get(endpoint,
                       params=params,
                       headers=headers)
   
    return response
        
def append_tweets(search_term, ending, num_queries, data_frame):
    
    remaining_calls = None
    reset_time = None
    
    for i in range(num_queries):
        end_temp = ending - timedelta(minutes = i*60)
        end_time = end_temp.strftime(dtformat)
        start_time = (end_temp - timedelta(minutes = 60)).strftime(dtformat)
        #print('start time = ', start_time)
        #print('end time = ', end_time)
        
        if (reset_time is not None and remaining_calls is not None 
            and remaining_calls < 1):
            
            sleep_time = int(reset_time) - int(time.time())
            if sleep_time > 0:
                log.warning(f"Rate limit reached. Sleeping for: {sleep_time}")
                time.sleep(sleep_time + 1)  # Sleep for an extra second
        
        response = gather_tweets(search_term,start_time,end_time)
        
        rem_calls = response.headers.get('x-rate-limit-remaining')
        if rem_calls is not None:
            remaining_calls = int(rem_calls)
        elif remaining_calls is not None:
            remaining_calls -= 1
            
        reset_time = response.headers.get('x-rate-limit-reset')
        if reset_time is not None:
            reset_time = int(reset_time)
            
        print(remaining_calls)
        
        for tweet in response.json()['data']:
            if tweet['text'][:2] != 'RT':
                row = get_data(tweet)
                data_frame = data_frame.append(row,ignore_index = True)
        
        if i<num_queries-1: time.sleep(1) #wait 1 second for each iteration to avoid overloading
        
    return data_frame
        

In [20]:
end = datetime.now() - timedelta(days = 1)
df = pd.DataFrame()
df = append_tweets('($tsla) (lang:en)', end, 400 ,df)

start time =  2021-06-21T12:02:59Z
end time =  2021-06-21T13:02:59Z


In [21]:
print(df.shape)
df.head()

(410, 3)


,created_at,id,text
0,2021-06-21T13:02:54.000Z,1406960821737988110,RT @EliteOptions2: $FVRR - Above 228 - Swing T...
1,2021-06-21T13:02:52.000Z,1406960811386589186,🔴LIVE Pre Market-Stock Updates-News \n6/21/202...
2,2021-06-21T13:02:49.000Z,1406960801785675786,RT @EliteOptions2: $DOCU - Above 271 - Trade I...
3,2021-06-21T13:02:44.000Z,1406960780528918534,RT @EliteOptions2: $SQ - Above 236 - Trade Ide...
4,2021-06-21T13:02:42.000Z,1406960770219483136,RT @TheStreet: The surprising thing that could...
